In [3]:
import time, os, threading, cv2, ast, re
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

In [ ]:
def scrape(path, headless=False):
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--hide-scrollbars")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(path)
    driver.maximize_window()
    time.sleep(1)
    
    # ==============================================================================================
#     # Complete login
    try:
        provided_u_name = "haider1805@icloud.com"
        user_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, "username")))   
        user_name.send_keys(provided_u_name)
        cont = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "nextButton")))   
        cont.click()
        time.sleep(1)

        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, "password")))
        password.send_keys(provided_pass)
        cont = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "loginBtn")))   
        cont.click()

        # login_tab = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "onelogin-main__btn")))
        # login_tab.click()
    except Exception as e:
        print(f"Login failed: {e}")
    
    # ==============================================================================================
    # Handle cookie consent banner
    try:
        cookie_accept = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/button[@id="onetrust-accept-btn-handler"]')))
        cookie_accept.click()
    except Exception as e:
        print(f"Cookie banner handling error: {e}")


    return driver

# def record_website_screen(driver, output_file="live_rec.avi", fps=7):
#     try:
#         img = driver.get_screenshot_as_png()
#         nparr = np.frombuffer(img, np.uint8)
#         frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#         height, width, _ = frame.shape
#         fourcc = cv2.VideoWriter_fourcc(*'XVID')
#         out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

#         frame_count = 0
#         while True:
#             img = driver.get_screenshot_as_png()
#             nparr = np.frombuffer(img, np.uint8)
#             frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#             frame_count += 1
#             out.write(frame)
#             time.sleep(1/fps)
#     except Exception as e:
        
#         print(f"Error in screen recording: {e}")
#     finally:
#         out.release()
#         print(f"Recording saved, captured {frame_count} frames.")

# def start_recording(driver):
#     thread = threading.Thread(target=record_website_screen, args=(driver,))
#     thread.daemon = True
#     thread.start()

def save_full_screenshot(driver, prefix="bidding_status"):
    try:
        os.makedirs("screenshots", exist_ok=True)
        filename = f"screenshots/{prefix}.png"
        driver.save_screenshot(filename)
        print(f"📸 Screenshot saved: {filename}")
    except Exception as e:
        print(f"❌ Screenshot failed: {e}")

if __name__ == "__main__":
    path = "https://www.bca.co.uk/sales/live?catalogueRef=COSV51"
    driver = scrape(path, headless=False)
    print("Starting screen recording...")
    # start_recording(driver)
    
    print("Starting data extraction...")
    results = []
    try:
        while True:
            if not driver.window_handles:
                print("Browser window closed. Stopping script.")
                break

            details = {}

            try:
                try:
                    ref = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Refresh Page"]')))
                    if ref:
                        ref.click()
                except:
                    print("No ref")
                    
                try:
                    reg = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//span[contains(@class,"Vrm__text")])[1]')))
                    if reg:
                        details['Reg'] = reg.get_attribute("textContent").strip()
                except:
                    print("No reg number")
                    
                data_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-gIEZMH ifBQUR Space sc-dOvA-dm eDtQbV BoxBase sc-bXWnss eQASzj Box StyledBoxElements__PanelContainer-sc-2z8npl-4 kCgoSs"]')))

                if data_card:
                    prices = WebDriverWait(data_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/span[@class="sc-gIEZMH cuHskl Space sc-cRmqLi fJJMDN Typography Typography--body-small"]')))
                    sale = WebDriverWait(data_card, 10).until(EC.presence_of_element_located((By.XPATH, './/span[@class="sc-gIEZMH cuHskl Space sc-cRmqLi haxHMn Typography Typography--body-small"]')))
                    details['Bidding History'] = [price.get_attribute("textContent").strip() for price in prices]
                    details['Bidding Status'] = sale.get_attribute("textContent").strip()
                    if details['Bidding Status']:
                        save_full_screenshot(driver, prefix=details['Reg'].replace(" ", "_"))
                    results.append(details)

          
                    df = pd.DataFrame(results, columns=['Reg' ,"Bidding History", "Bidding Status"])
                    df.to_csv("live_data.csv", index=False)

                else:
                    print("No data card found.")

                time.sleep(3) 
                
            except Exception as e:
                print(f"Error while extracting data:")
                    
    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        driver.quit()
        print("Browser closed.")


Cookie banner handling error: Message: 
Stacktrace:
	GetHandleVerifier [0x0x108ba83+63395]
	GetHandleVerifier [0x0x108bac4+63460]
	(No symbol) [0x0xed2113]
	(No symbol) [0x0xf1a85e]
	(No symbol) [0x0xf1abfb]
	(No symbol) [0x0xf62f92]
	(No symbol) [0x0xf3f3f4]
	(No symbol) [0x0xf607ba]
	(No symbol) [0x0xf3f1a6]
	(No symbol) [0x0xf0e7b2]
	(No symbol) [0x0xf0f654]
	GetHandleVerifier [0x0x1308883+2672035]
	GetHandleVerifier [0x0x1303cba+2652634]
	GetHandleVerifier [0x0x10b2bca+223466]
	GetHandleVerifier [0x0x10a2cb8+158168]
	GetHandleVerifier [0x0x10a978d+185517]
	GetHandleVerifier [0x0x1093b78+96408]
	GetHandleVerifier [0x0x1093d02+96802]
	GetHandleVerifier [0x0x107e90a+9770]
	BaseThreadInitThunk [0x0x766a7ba9+25]
	RtlInitializeExceptionChain [0x0x76f4c3ab+107]
	RtlClearBits [0x0x76f4c32f+191]

Starting screen recording...
Starting data extraction...
No ref
📸 Screenshot saved: screenshots/YP10_PBZ_20251029_140116.png
No ref
📸 Screenshot saved: screenshots/YP10_PBZ_20251029_140122.png
No r

In [ ]:
# def scrape(path, headless=False):
#     chrome_options = Options()
#     if headless:
#         chrome_options.add_argument("--headless")
#     chrome_options.add_argument("--window-size=1920x1080")
#     chrome_options.add_argument("--disable-gpu")
#     chrome_options.add_argument("--no-sandbox")
#     chrome_options.add_argument("--disable-dev-shm-usage")
#     chrome_options.add_argument("--hide-scrollbars")
    
#     service = Service(ChromeDriverManager().install())
#     driver = webdriver.Chrome(service=service, options=chrome_options)
#     driver.get(path)
#     driver.maximize_window()
#     time.sleep(1)
    
#     # ==============================================================================================
# #     # Complete login
#     try:
#         provided_u_name = "haider1805@icloud.com"
#         user_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, "username")))   
#         user_name.send_keys(provided_u_name)
#         cont = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "nextButton")))   
#         cont.click()
#         time.sleep(1)

#         provided_pass = "Muhssan7865"
#         password = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, "password")))
#         password.send_keys(provided_pass)
#         cont = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "loginBtn")))   
#         cont.click()

#         # login_tab = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "onelogin-main__btn")))
#         # login_tab.click()
#     except Exception as e:
#         print(f"Login failed: {e}")
    
#     # ==============================================================================================
#     # Handle cookie consent banner
#     try:
#         cookie_accept = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/button[@id="onetrust-accept-btn-handler"]')))
#         cookie_accept.click()
#     except Exception as e:
#         print(f"Cookie banner handling error: {e}")


#     return driver

# def record_website_screen(driver, output_file="live_rec.avi", fps=7):
#     try:
#         img = driver.get_screenshot_as_png()
#         nparr = np.frombuffer(img, np.uint8)
#         frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#         height, width, _ = frame.shape
#         fourcc = cv2.VideoWriter_fourcc(*'XVID')
#         out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

#         frame_count = 0
#         while True:
#             img = driver.get_screenshot_as_png()
#             nparr = np.frombuffer(img, np.uint8)
#             frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#             frame_count += 1
#             out.write(frame)
#             time.sleep(1/fps)
#     except Exception as e:
        
#         print(f"Error in screen recording: {e}")
#     finally:
#         out.release()
#         print(f"Recording saved, captured {frame_count} frames.")

# def start_recording(driver):
#     thread = threading.Thread(target=record_website_screen, args=(driver,))
#     thread.daemon = True
#     thread.start()

# if __name__ == "__main__":
#     path = "https://www.bca.co.uk/sales/live?catalogueRef=COSV49"
#     driver = scrape(path, headless=False)
#     print("Starting screen recording...")
#     start_recording(driver)
    
#     print("Starting data extraction...")
#     results = []
#         # ==============================================================================================
#     # Start continuous scraping loop
#     try:
#         while True:
#             if not driver.window_handles:
#                 print("Browser window closed. Stopping script.")
#                 break

#             details = {}

#             try:
#                 try:
#                     ref = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Refresh Page"]')))
#                     if ref:
#                         ref.click()
#                 except:
#                     print("No ref")
                    
#                 try:
#                     reg = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//span[contains(@class,"Vrm__text")])[1]')))
#                     if reg:
#                         details['Reg'] = reg.get_attribute("textContent").strip()
#                 except:
#                     print("No reg number")
                    
#                 data_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-gIEZMH ifBQUR Space sc-dOvA-dm eDtQbV BoxBase sc-bXWnss eQASzj Box StyledBoxElements__PanelContainer-sc-2z8npl-4 kCgoSs"]')))

#                 if data_card:
#                     # car_title = WebDriverWait(data_card, 10).until(EC.presence_of_element_located((By.XPATH, './/span[@class="sc-gIEZMH htFYJm Space sc-cRmqLi iZXkIg Typography Typography--body-small"]')))
#                     # reg = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/span[@class="sc-gIEZMH cuHskl Space sc-cRmqLi kKmHGu Typography Typography--body Vrm__text"]')))
#                     prices = WebDriverWait(data_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/span[@class="sc-gIEZMH cuHskl Space sc-cRmqLi fJJMDN Typography Typography--body-small"]')))
#                     sale = WebDriverWait(data_card, 10).until(EC.presence_of_element_located((By.XPATH, './/span[@class="sc-gIEZMH cuHskl Space sc-cRmqLi haxHMn Typography Typography--body-small"]')))

#                     # details['Title'] = car_title.get_attribute("textContent").strip()
#                     # details['Reg'] = reg.get_attribute("textContent").strip()
#                     details['Bidding History'] = [price.get_attribute("textContent").strip() for price in prices]
#                     details['Bidding Status'] = sale.get_attribute("textContent").strip()

#                     results.append(details)

#                     # Convert results to DataFrame and save to CSV
#                     df = pd.DataFrame(results, columns=['Reg' ,"Bidding History", "Bidding Status"])
#                     df.to_csv("live_data.csv", index=False)

#                 else:
#                     print("No data card found.")

#                 time.sleep(3)  # Wait before checking for updates
                
#             except Exception as e:
#                 print(f"Error while extracting data:")
                    
#     except KeyboardInterrupt:
#         print("Stopped by user.")
#     finally:
#         driver.quit()
#         print("Browser closed.")


In [ ]:
df = pd.read_csv("live_data.csv")

df2 = df.drop_duplicates(subset=['Reg'], keep='last')

finalized_df = df2.to_csv("live_data_CLEANED.csv", index=False)
# delete the original df
os.remove("live_data.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 48: invalid start byte

In [ ]:
def clean_prices(data):
    return (data.replace("£", ' ').replace(",", ""))

df['Last Bid'] = df['Bidding History'].apply(lambda x : clean_prices(ast.literal_eval(x)[0]))
df.to_csv("live_data_separate_prices.csv", index=False)

top_car_df = pd.read_csv("live_data_separate_prices.csv")
top_car_live_cleaned_df = pd.read_csv("BCA_json.csv")

merged_df = top_car_live_cleaned_df.merge(top_car_df[['Reg',"Bidding Status" , "Bidding History", "Last Bid"]], on='Reg', how='left')

# Save the updated dataset
output_path = "bca_live_data_merged_final.csv"
merged_df.to_csv(output_path, index=False)

# os.remove("live_data_CLEANED.csv")
# os.remove("BCA_json.csv")
# os.remove("live_data_separate_prices.csv")

NameError: name 'df' is not defined

In [ ]:
# Load files
df1 = pd.read_csv("bca_live_data_merged_final.csv")
df2 = pd.read_csv("Vars.csv")

df2 = df2[['Make', 'Model', 'Variant']]

# Convert to uppercase for case-insensitive matching
df1['Make'] = df1['Make'].str.upper()
df1['Model'] = df1['Model'].str.upper()
df2['Make'] = df2['Make'].str.upper()
df2['Model'] = df2['Model'].str.upper()
df2['Derivative'] = df2['Variant'].str.upper()

# Function to correct model and variant
def correct_model_variant(row):
    make = row['Make']
    model = row['Model']
    
    # Case 1: Exact match
    match_df = df2[(df2['Make'] == make) & (df2['Model'] == model)]
    if not match_df.empty:
        return pd.Series([model, row['Derivative']])
    
    # Case 2: Model found as variant
    match_variant_df = df2[(df2['Make'] == make) & (df2['Derivative'] == model)]
    if not match_variant_df.empty:
        corrected_model = match_variant_df.iloc[0]['Model']
        return pd.Series([corrected_model, row['Model']])
    
    # Case 3: No match at all
    return pd.Series([row['Model'], row['Derivative']])

# Apply correction logic
df1[['Corrected Model', 'Corrected Variant']] = df1.apply(correct_model_variant, axis=1)

# Case 4: Create 'New Variant' column
def create_new_variant(row):
    variant = str(row['Derivative']) if pd.notna(row['Derivative']) else 'Derivative'
    corrected_variant = str(row['Corrected Variant']) if pd.notna(row['Corrected Variant']) else 'Model'
    
    if variant == corrected_variant:
        return variant
    else:
        return f"{corrected_variant} {variant}"

df1['New Variant'] = df1.apply(create_new_variant, axis=1)

# Save the final output
df1.to_csv('bca_merged_and_Corrected.csv', index=False)
# os.remove("bca_live_data_merged_final.csv")

AttributeError: Can only use .str accessor with string values!

In [ ]:
df1 = pd.read_csv("Vars.csv")
df1 = df1[['Make', 'Model', 'Variant']]
df2 = pd.read_csv("bca_merged_and_Corrected.csv")

def clean(text):
    if pd.isna(text):
        return ""
    return str(text).strip().upper()

# Function to extract variant
def extract_variant(row, df1):
    model = clean(row['Corrected Model'])
    Variant = clean(row['New Variant'])

    if not model or not Variant:
        return None

    # Get all variants for that model
    model_variants = df1[df1['Model'].str.upper().str.strip() == model]['Variant']

    for variant in model_variants:
        variant = clean(variant)
        pattern = r'\b' + re.escape(variant) + r'\b'
        if re.search(pattern, Variant, re.IGNORECASE):
            return variant

    return None
# Apply and get matches
df2['Variant'] = df2.apply(lambda row: extract_variant(row, df1), axis=1)
df2.to_csv("bca_merged_variants_final.csv")
# os.remove("bca_merged_and_Corrected.csv")

In [ ]:
df = pd.read_csv("bca_merged_variants_final.csv")

df2 = df.drop_duplicates(subset=['Reg'], keep='last')
df2 = df2.sort_values(by='Make', ascending=True)

finalized_df = df2.to_csv("bca_merged_variants_final_cleaned.csv", index=False)
# os.remove("bca_merged_variants_final.csv")

EmptyDataError: No columns to parse from file

In [ ]:
df = pd.read_csv("bca_merged_variants_final_cleaned.csv")

df2 = df.drop_duplicates(subset=['Reg'], keep='last')
df2 = df2.sort_values(by='Make', ascending=True)

finalized_df = df2.to_csv("Final_BCA.csv", index=False)
# os.remove("bca_merged_variants_final_cleaned.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'bca_merged_variants_final_cleaned.csv'